## Importar librerías

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [3]:
from sklearn.cluster import KMeans
from sklearn.cluster import DBSCAN
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA

In [4]:
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MaxAbsScaler
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import MinMaxScaler

In [5]:
from ipywidgets import interact

In [6]:
plt.rcParams['figure.figsize'] = (12,8)

# Plots & distributions

In [7]:
df = pd.read_csv('./Data/data_nba.csv', index_col=0)

La celda de abajo la dejo como 'Raw NBConvert' porque si no ocupa mucha memoria

# Data wrangling

Restamos los triples a los tiros de campo para no duplicar información

In [8]:
df['FG'] = df['FG'] - df['FG3']
df['FGA'] = df['FGA'] - df['FG3A']

Vemos como ahora los tiros de campo x2 + los triples x3 + los tiros libres x1 siempre son iguales a los puntos

In [9]:
(df['FG']*2 + df['FG3']*3 + df['FT'] == df['PTS']).value_counts()

True    539422
dtype: int64

Eliminamos las columnas de los porcentajes porque las recalcularemos luego tras restar los triples a los tiros de campo

In [10]:
df.drop(['FG_PCT', 'FG3_PCT', 'FT_PCT'], axis=1, inplace=True)

In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 539422 entries, 0 to 539421
Data columns (total 32 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   date        539422 non-null  object 
 1   team        539422 non-null  object 
 2   player      539422 non-null  object 
 3   role        539422 non-null  int64  
 4   MP          539422 non-null  float64
 5   FG          539422 non-null  int64  
 6   FGA         539422 non-null  int64  
 7   FG3         539422 non-null  int64  
 8   FG3A        539422 non-null  int64  
 9   FT          539422 non-null  int64  
 10  FTA         539422 non-null  int64  
 11  ORB         539422 non-null  int64  
 12  DRB         539422 non-null  int64  
 13  AST         539422 non-null  int64  
 14  STL         539422 non-null  int64  
 15  BLK         539422 non-null  int64  
 16  TOV         539422 non-null  int64  
 17  PF          539422 non-null  int64  
 18  PTS         539422 non-null  int64  
 19  PL

## Groupby player & season - data per game

Creamos un diccionario que tenga un df con la suma de los datos de campo de cada jugador para cada temporada pero mostrando los datos por partido, para que al hacer clustering se quede únicamente con el estilo de juego de cada jugador más que con el acierto

In [12]:
field_columns = ['player', 'MP', 'FG', 'FGA', 'FG3', 'FG3A', 'FT',
        'FTA', 'ORB', 'DRB','AST', 'STL', 'BLK', 'TOV', 'PF', 'PLUS_MINUS']

In [13]:
other_columns = ['player', 'team', 'season_end', 'salary', 'height', 'weight', 'age',
       'pos_PG', 'pos_C', 'pos_PF', 'pos_SG', 'pos_SF', 'hand_right',
       'hand_left']

In [14]:
stats_pg = dict()
for y in range(2001,2022):
    stats_pg[y] = df[df['season_end']==y].groupby('player')[field_columns].sum().merge(df[df['season_end']==y][other_columns],
                                        on='player', how='left').drop_duplicates('player').reset_index(drop=True)

In [15]:
stats_pg[2001].columns

Index(['player', 'MP', 'FG', 'FGA', 'FG3', 'FG3A', 'FT', 'FTA', 'ORB', 'DRB',
       'AST', 'STL', 'BLK', 'TOV', 'PF', 'PLUS_MINUS', 'team', 'season_end',
       'salary', 'height', 'weight', 'age', 'pos_PG', 'pos_C', 'pos_PF',
       'pos_SG', 'pos_SF', 'hand_right', 'hand_left'],
      dtype='object')

In [16]:
# Columnas que no usaremos en el clustering

non_clustering_columns = ['player', 'hand_right', 'hand_left', 'MP','FG','FGA','FG3','FG3A','FT','FTA',
                      'team','season_end','salary','age','PF','PLUS_MINUS']

In [17]:
stats_pg[2001].columns

Index(['player', 'MP', 'FG', 'FGA', 'FG3', 'FG3A', 'FT', 'FTA', 'ORB', 'DRB',
       'AST', 'STL', 'BLK', 'TOV', 'PF', 'PLUS_MINUS', 'team', 'season_end',
       'salary', 'height', 'weight', 'age', 'pos_PG', 'pos_C', 'pos_PF',
       'pos_SG', 'pos_SF', 'hand_right', 'hand_left'],
      dtype='object')

In [18]:
for y in range(2001,2022):
    
    # Primero filtramos los jugadores con menos de 100 minutos ya que pueden mostrar resultados engañosos por partido
    stats_pg[y] = stats_pg[y][stats_pg[y]['MP'] >= 100].reset_index(drop=True)
    
    # Porcentaje de tiros de campo
    stats_pg[y]['FG_pc'] = stats_pg[y]['FG'] / stats_pg[y]['FGA']
    
    # Porcentaje de triples
    stats_pg[y]['FG3_pc'] = stats_pg[y]['FG3'] / stats_pg[y]['FG3A']
    
    # Porcentaje de tiros libres
    stats_pg[y]['FT_pc'] = stats_pg[y]['FT'] / stats_pg[y]['FTA']
    
    # Rebotes ofensivos por partido
    stats_pg[y]['ORB'] = stats_pg[y]['ORB'] / stats_pg[y]['MP'] * 48

    # Rebotes defensivos por partido
    stats_pg[y]['DRB'] = stats_pg[y]['DRB'] / stats_pg[y]['MP'] * 48

    # Asistencias por partido
    stats_pg[y]['AST'] = stats_pg[y]['AST'] / stats_pg[y]['MP'] * 48
    
    # Robos por partido
    stats_pg[y]['STL'] = stats_pg[y]['STL'] / stats_pg[y]['MP'] * 48
    
    # Tapones por partido
    stats_pg[y]['BLK'] = stats_pg[y]['BLK'] / stats_pg[y]['MP'] * 48
    
    # Recuperaciones por partido
    stats_pg[y]['TOV'] = stats_pg[y]['TOV'] / stats_pg[y]['MP'] * 48
    
    # Plus_minus por partido
    stats_pg[y]['PLUS_MINUS'] = stats_pg[y]['PLUS_MINUS'] / stats_pg[y]['MP'] * 48
    
    # Rellenamos los porcentajes NaN con ceros
    stats_pg[y].fillna(0, inplace=True)

# Clustering

Intentamos hacer clustering sólo con las estadísticas de campo y luego compararemos para ver si se corresponde con otras como la altura, posición, etc.

In [20]:
def clustering_pg(diccionario, scaler=StandardScaler(), modelo_clust=KMeans(5, random_state=42)):
    """Esta función aplicará escalado (por defecto StandardScaler) y clustering
    (por defecto KMeans de 5 clusters) a los datasets
    de cada uno de los años dentro del diccionario 'stats_pg' """
    
    # Iteramos para que lo haga con cada uno de los años
    for year in diccionario:
        
        # Primero escalamos cada uno de los dataframes con las 'field columns' pero quitándole la variable 'player'
        data_scaled = scaler.fit_transform(diccionario[year].drop(non_clustering_columns, axis=1))
        
        # Ahora asignamos las labels a cada dataframe dentro del diccionario
        diccionario[year]['labels'] = modelo_clust.fit(data_scaled).labels_
  

In [21]:
clustering_pg(stats_pg)

Una vez hecho el clustering, volvemos a introducir la variable 'PTS' en el dataframe, ya que nos servirá a la hora de hacer visualizaciones

In [23]:
for y in range(2001,2022):
    
    # Puntos por partido
    stats_pg[y]['PTS'] = (stats_pg[y]['FG']*2 + stats_pg[y]['FG3']*3 + stats_pg[y]['FT']) / stats_pg[y]['MP'] * 48

## Visualización clustering

Creamos una función que nos permita visualizar el clustering en función del año. Hemos utilizado TSNE() porque es con la que obtenemos mejores resultados a la hora de visualizar la separación de clusters.

In [37]:
@interact(year=range(2001,2022))

def viz_clusters_pg(year):
    """Esta función nos permite visualizar el clustering realizado anteriormente
    a partir de un diccionario con dataframes"""
    
    data = stats_pg[year].drop(non_clustering_columns, axis=1)
    pc = PCA().fit_transform(data)
    test = pd.DataFrame(pc[:,:2], columns=['t1', 't2'])
    test['lb'] = stats_pg[year]['labels'].apply(lambda x: str(x))
    sns.scatterplot(data=test, x='t1', y='t2', hue='lb')

interactive(children=(Dropdown(description='year', options=(2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 20…

# Data viz

## Players - data per game

Exportamos el dataframe con nuestros datos por partido para hacer visualizaciones

In [25]:
data_exported_pg = pd.DataFrame()
for y in range(2001,2022):
    data_exported_pg = data_exported_pg.append(stats_pg[y])

Recuperamos la variable puntos para visualización

No hay valores nulos y los porcentajes de tiro ya están añadidos, con lo que sólo queda resetear índice y exportar a csv

In [26]:
data_exported_pg.reset_index(drop=True, inplace=True)

In [27]:
data_exported_pg.to_csv('./Data/data_exported_pg.csv')

## Game statistics

Creamos un diccionario con el número de partidos de cada una de las temporadas regulares

In [28]:
# Usamos drop_duplicates() con cada hora exacta y equipo y lo dividimos entre 2

nrgames = dict(zip(range(2001,2022),[int(len(df[df['season_end']==year].drop_duplicates(['date', 'team']))/2) for year in range(2001,2022)]))


In [29]:
df.drop_duplicates().groupby('season_end').sum()[['MP', 'FG', 'FGA', 'FG3', 'FG3A', 'FT',
        'FTA', 'ORB', 'DRB','AST', 'STL', 'BLK', 'TOV', 'PF']]

,MP,FG,FGA,FG3,FG3A,FT,FTA,ORB,DRB,AST,STL,BLK,TOV,PF
season_end,,,,,,,,,,,,,,
2001,599002.716667,76408,165852,12147,34285,46140,61692,29803,75616,53998,19401,13106,35586,55595
2002,592729.883333,76151,163671,12895,36578,44260,58914,30069,74372,53925,19121,12861,33958,52304
2003,601696.883333,76260,164922,12882,36778,46418,61249,30264,75820,53606,19726,12550,35588,54606
2004,593141.633333,73119,159209,12733,36818,44757,59654,29661,74337,52294,19444,12538,34965,52911
2005,625872.200000,78615,166995,14558,40976,50932,67464,31188,77490,54891,19437,12837,35830,58854
2006,629933.783333,78244,163398,15071,42139,51091,68508,28976,77774,53305,18587,12264,35701,59557
2007,631021.616667,78940,162874,16060,44816,51051,67944,28982,78162,55297,18955,12080,37752,58207
2008,625840.683333,79184,163822,17212,47554,48812,64515,28714,79557,56625,18872,12136,35032,54740
2009,616308.416667,77772,160173,17008,46492,48818,63407,28252,77578,53440,18566,12289,34230,54120


In [30]:
gamestats = df.drop_duplicates().groupby('season_end').sum()[['MP', 'FG', 'FGA', 'FG3', 'FG3A', 'FT',
        'FTA', 'ORB', 'DRB','AST', 'STL', 'BLK', 'TOV', 'PF']]

In [31]:
gamestats['PTS'] = gamestats['FG']*2 + gamestats['FG3']*3 + gamestats['FT']

In [32]:
gamestats

,MP,FG,FGA,FG3,FG3A,FT,FTA,ORB,DRB,AST,STL,BLK,TOV,PF,PTS
season_end,,,,,,,,,,,,,,,
2001,599002.716667,76408,165852,12147,34285,46140,61692,29803,75616,53998,19401,13106,35586,55595,235397
2002,592729.883333,76151,163671,12895,36578,44260,58914,30069,74372,53925,19121,12861,33958,52304,235247
2003,601696.883333,76260,164922,12882,36778,46418,61249,30264,75820,53606,19726,12550,35588,54606,237584
2004,593141.633333,73119,159209,12733,36818,44757,59654,29661,74337,52294,19444,12538,34965,52911,229194
2005,625872.200000,78615,166995,14558,40976,50932,67464,31188,77490,54891,19437,12837,35830,58854,251836
2006,629933.783333,78244,163398,15071,42139,51091,68508,28976,77774,53305,18587,12264,35701,59557,252792
2007,631021.616667,78940,162874,16060,44816,51051,67944,28982,78162,55297,18955,12080,37752,58207,257111
2008,625840.683333,79184,163822,17212,47554,48812,64515,28714,79557,56625,18872,12136,35032,54740,258816
2009,616308.416667,77772,160173,17008,46492,48818,63407,28252,77578,53440,18566,12289,34230,54120,255386


In [33]:
for i,c in gamestats.iterrows():
    
    gamestats.loc[i] = gamestats.loc[i] / nrgames[i]


In [34]:
gamestats

,MP,FG,FGA,FG3,FG3A,FT,FTA,ORB,DRB,AST,STL,BLK,TOV,PF,PTS
season_end,,,,,,,,,,,,,,,
2001,485.022443,61.868826,134.293117,9.835628,27.761134,37.360324,49.953036,24.131984,61.227530,43.723077,15.709312,10.612146,28.814575,45.016194,190.604858
2002,481.111918,61.810877,132.849838,10.466721,29.689935,35.925325,47.819805,24.406656,60.366883,43.770292,15.520292,10.439123,27.563312,42.454545,190.947240
2003,486.416236,61.649151,133.324171,10.413905,29.731609,37.524656,49.514147,24.465643,61.293452,43.335489,15.946645,10.145513,28.769604,44.143897,192.064673
2004,480.665829,59.253647,129.018639,10.318476,29.836305,36.269854,48.341977,24.036467,60.240681,42.377634,15.756888,10.160454,28.334684,42.877634,185.732577
2005,485.925621,61.036491,129.654503,11.302795,31.813665,39.543478,52.378882,24.214286,60.163043,42.617236,15.090839,9.966615,27.818323,45.694099,195.524845
2006,488.699599,60.701319,126.763382,11.692009,32.691234,39.636152,53.148177,22.479441,60.336695,41.353763,14.419705,9.514352,27.696664,46.204034,196.114818
2007,494.531048,61.865204,127.644201,12.586207,35.122257,40.008621,53.247649,22.713166,61.255486,43.336207,14.855016,9.467085,29.586207,45.616771,201.497649
2008,490.086674,62.007831,128.286609,13.478465,37.238841,38.223962,50.520752,22.485513,62.299922,44.342208,14.778387,9.503524,27.433046,42.866092,202.675020
2009,484.138583,61.093480,125.823252,13.360566,36.521603,38.348782,49.809112,22.193244,60.941084,41.979576,14.584446,9.653574,26.889238,42.513747,200.617439


Reseteamos índice y exportamos

In [35]:
gamestats.reset_index().to_csv('./Data/gamestats.csv')